In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import matplotlib.pyplot as plt 

from glob import glob 

from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Dense,Dropout,Flatten
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import BatchNormalization
#from keras.callbacks import (ModelCheckpoint,ReduceLROnPlateau,CSVLogger)
from sklearn import preprocessing

from sklearn.model_selection import train_test_split
import cv2

import numpy as np 
import pandas as pd


from keras import layers
from keras import models
from keras import optimizers

from keras.preprocessing.image import img_to_array, load_img
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D,GlobalMaxPool2D
from keras.layers. normalization import BatchNormalization
from keras.optimizers import Adam

# Read and explore the given dataset

In [ ]:
TRAIN_DATA_DIR = '/kaggle/input/plant-seedlings-classification/train/*/*.png'
TEST_DATA_DIR = '/kaggle/input/plant-seedlings-classification/test/*.png'

In [ ]:
data = glob(TRAIN_DATA_DIR)

training_data=[]
training_label=[]

data_count= len(data)

print("Reading Data")
for d in data:
    training_data.append(cv2.resize(cv2.imread(d),(240,240)))
    training_label.append(d.split('/')[-2])
print("Data Read Complete")
training_data = np.asarray(training_data)
training_label = pd.DataFrame(training_label)

In [ ]:
training_data

In [ ]:
print("Size Of traning Data",len(training_data))

In [ ]:
training_label[0][1]

In [ ]:
plt.imshow(training_data[1])

In [ ]:
training_data[1].shape

In [ ]:
training_label[0].unique()

In [ ]:

category = ['Scentless Mayweed', 'Common wheat', 'Charlock', 'Black-grass',
       'Sugar beet', 'Loose Silky-bent', 'Maize', 'Cleavers',
       'Common Chickweed', 'Fat Hen', 'Small-flowered Cranesbill',
       'Shepherds Purse']
print_data = {}

print('------ Data Contains----------')

for s in category:
    count= len(os.listdir(os.path.join('../input/plant-seedlings-classification/train',s )))
    print('{} data for - {} category'.format(count,s))
    print_data[s] = count
    
plt.figure(figsize=(23, 8))  
sns.barplot(list(print_data.keys()), list(print_data.values()))

In [ ]:
LabelEncode = preprocessing.LabelEncoder()
LabelEncode.fit(training_label[0])
new_label = LabelEncode.transform(training_label[0])
clearalllabels = np_utils.to_categorical(new_label)

print(training_data.shape, clearalllabels.shape)

In [ ]:
training_data[1][1]# Before Normilization

# Split the data

In [ ]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(training_data,clearalllabels,test_size=0.2,random_state=1,stratify=clearalllabels)

print(x_train.shape,x_test.shape,y_train.shape,y_test.shape)

In [ ]:
x_train.shape[1:]  #Dimensions of the image

In [ ]:
# Normalization of the Image Data
x_train = x_train.astype('float32') / 255
x_test = x_test.astype('float32') / 255 

In [ ]:
plt.imshow(x_train[0])

In [ ]:
data_generator = ImageDataGenerator(rotation_range=20,
                                    zoom_range=0.2,
                                    width_shift_range=0.2,
                                    height_shift_range=0.2,
                                    horizontal_flip=True,
                                    vertical_flip=True,
    )

In [ ]:
data_generator.fit(x_train) 

In [ ]:
#Augmented Images
from matplotlib import pyplot as plt
gen = data_generator.flow(x_train[0:1], batch_size=1)
for i in range(1, 6):
    plt.subplot(1,5,i)
    plt.axis("off")
    plt.imshow(gen.next().squeeze(), cmap='gray')
    plt.plot()
plt.show()

# Build the Model and summarize the model

In [ ]:
import tensorflow as tf

In [ ]:
tf.keras.backend.clear_session()
#initialising sequential model
model = Sequential()
#adding 1st convolution layer with 64 filter and imput shape 240 x240x3 with relu function
model.add(Conv2D(filters=64, kernel_size=(3, 3), input_shape=(240, 240, 3), activation='relu'))
#normalising batch
model.add(BatchNormalization(axis=3))
#adding 2nd convolution layer with 64 filters
model.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu'))
#maxpooling
model.add(MaxPooling2D((2, 2)))
#normalising batch
model.add(BatchNormalization(axis=3))
model.add(Dropout(0.1))
#adding 3rd convolution layer with 32 filters
model.add(Conv2D(filters=32, kernel_size=(3, 3), activation='relu'))
#normalising batch
model.add(BatchNormalization(axis=3))
#adding 4th convolution layer with 32 filters
model.add(Conv2D(filters=32, kernel_size=(3, 3), activation='relu'))
#maxpooling
model.add(MaxPooling2D((2, 2)))
#normalising batch
model.add(BatchNormalization(axis=3))
model.add(Dropout(0.1))
# Global Max pooling layer
model.add(GlobalMaxPool2D())

#flattening layer
model.add(Flatten())
#Dense layer
#adding 1 st dense layer
model.add(Dense(100, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))
#adding 2 nd dense layer
model.add(Dense(50, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))
#adding dense layer with same output as no of cateogries, in our case 12 category with softmax function
model.add(Dense(12, activation='softmax'))


In [ ]:
model.summary()

In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
#Saving the best model using model checkpoint callback
model_checkpoint=tf.keras.callbacks.ModelCheckpoint('plat-seedling.h5', 
                                                    save_best_only=True, 
                                                    monitor='val_accuracy', 
                                                    mode='max',
                                                    verbose=1)

In [ ]:
model_history=model.fit_generator(data_generator.flow(x_train,y_train,batch_size=16),epochs=200,
          validation_data=(x_test,y_test),callbacks=model_checkpoint)

# Evaluate the Model

In [ ]:
#Final score and accuracy of the model
score, acc = model.evaluate(x_test,y_test)
score1, acc1 = model.evaluate(x_train,y_train)
print('Test score:', score,'   Test accuracy:', acc)
print('Train score:', score1,'   Train accuracy:',acc1)


In [ ]:
from matplotlib import pyplot

#plotting the accuracy and the loss curve 

pyplot.plot(model_history.history['accuracy'], label='train')
pyplot.plot(model_history.history['val_accuracy'], label='test')
pyplot.legend()
pyplot.show()

pyplot.plot(model_history.history['loss'], label='train')
pyplot.plot(model_history.history['val_loss'], label='test')
pyplot.legend()
pyplot.show()


In [ ]:

path_to_test = TEST_DATA_DIR
pics = glob(path_to_test)
#creating two list for isnseritng testing and testing labels
testimages = []
tests = []
num = len(pics)
print(pics)
for i in pics:
    tests.append(i.split('/')[-1])
    testimages.append(cv2.resize(cv2.imread(i),(240,240)))
testimages= np.asarray(testimages)

In [ ]:
# Normalization of the Image Data
testimages = testimages.astype('float32') / 255

In [ ]:
plt.imshow(testimages[0])

# Predict the Result

In [ ]:
#using the model to predict the values on the testing data
prediction = model.predict(testimages)

pred = np.argmax(prediction,axis=1)
predStr = LabelEncode.classes_[pred]
submission = {'file':tests,'species':predStr}
submission = pd.DataFrame(submission)
submission.to_csv("Prediction.csv",index=False)

In [ ]:
submission.head()

In [ ]:
# serialize model to JSON
model_json = model.to_json()
with open("plant_seedling.json", "w") as json_file:
    json_file.write(model_json)

# serialize weights to HDF5
model.save_weights("plant-seedling.h5")

print("Saved model to disk")